In [1]:
from DataCollector import Twitter
import pandas as pd
from tqdm.notebook import tqdm
from decouple import config

tweets = Twitter.Tweets(config('TWITTER_API_KEY'),
               config('TWITTER_API_SECRET'),
               config('TWITTER_ACCESS_TOKEN_KEY'),
               config('TWITTER_ACCESS_TOKEN_SECRET'))


languages = {
                1: 'en',
                2: 'es',
                3: 'fr',
                4: 'de',
            }

In [2]:
for i in tqdm([0,1,2]):
    df = pd.read_csv('Data/FilteredTwitterIDs/' + str(i) + '.csv')
    ids = df.tweet_id.values.tolist()
    for j in tqdm(languages):
        for k in range(1,11):
            tweets.addTweets(ids[((j-1)*1000)+(k-1)*100:((j-1)*1000)+k*100])
        tweets.saveJSON('Data/Text/temp')
        if j != 1:
            tweets.translate(language=languages[j])
        tweets.saveJSON('Data/Text/NotProcessed/' + str(i) + str(j))
        tweets.reset()
        tweets.loadJSON('Data/Text/temp')
        tweets.preProcess(languages[j])
        if j != 1:
            tweets.translate(language=languages[j])
        tweets.saveJSON('Data/Text/Processed/' + str(i) + str(j))
        tweets.reset()



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]